In [ ]:
import manganite
%load_ext manganite

# Facility Location Problem

In [ ]:
from math import sqrt
import gurobipy as gp
from gurobipy import GRB
import seaborn as sns 
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd 
from itertools import product
import networkx as nx
import networkx.generators.small as gs
import matplotlib.pyplot as plt
import plotly.express as px
import geopandas as gpd
from numpy.random import randint, binomial, uniform
import string
import random
import plotly.graph_objects as go

In [ ]:
%%mnn widget --type table --var FACILITIES --tab Inputs --header Facilities
FACILITIES = pd.read_csv("./Facilities.csv")

In [ ]:
%%mnn widget --type table --var CUSTOMERS --tab Inputs --header Customers
CUSTOMERS = pd.read_csv("./Customers.csv")

In [ ]:
#Parameter setting
customers = CUSTOMERS.set_index('Customer').drop(columns=['demand']).T.to_dict('list')
facilities = FACILITIES.set_index('Facility').drop(columns=['opening_cost', 'capacity', 'cost_per_km']).T.to_dict('list')
opening_cost = FACILITIES['opening_cost'].to_list()
cost_per_km = FACILITIES['cost_per_km'].dropna()[0]
demand = CUSTOMERS['demand'].to_list()
capacity = FACILITIES['capacity'].to_list()

In [ ]:
# Compute key parameters of MIP model formulation
FACILITIES_ = pd.DataFrame.from_dict(facilities, orient='index', columns =['x_fac', 'y_fac'])
CUSTOMERS_ = pd.DataFrame.from_dict(customers, orient='index', columns =['x_cus', 'y_cus'])
lon_fac = FACILITIES_.x_fac
lat_fac = FACILITIES_.y_fac
lon_cus = CUSTOMERS_.x_cus
lat_cus = CUSTOMERS_.y_cus

num_facilities = len(FACILITIES_)
num_customers = len(CUSTOMERS_)
CF = list(product(range(num_customers), range(num_facilities))) #cartesian product for indexing

#Lists for dictionaries for plotting
fac_key_list=list(facilities.keys())
fac_val_list=list(facilities.values())
cus_key_list=list(customers.keys())
cus_val_list=list(customers.values())

# Compute distance between facility and customer
## This function determines the Euclidean distance between a facility and customer sites.
def distance_computation(location_1, location_2):
    d_x = location_1[0] - location_2[0]
    d_y = location_1[1] - location_2[1]
    return sqrt(d_x*d_x + d_y*d_y)
DISTANCE = np.array([distance_computation(list(customers.values())[c], list(facilities.values())[f]) for c, f in CF]).reshape(num_customers,num_facilities)

In [ ]:
%%mnn execute --on button Optimize --returns shipments
# MIP  model formulation
def check_status(status):
    if status == GRB.OPTIMAL:
        return
    elif status == GRB.INF_OR_UNBD:
        raise RuntimeError('Model is infeasible or unbounded')
    elif status == GRB.INFEASIBLE:
        raise RuntimeError('Model is infeasible')
    elif status == GRB.UNBOUNDED:
        raise RuntimeError('Model is unbounded')
    else:
        raise RuntimeError('Optimization ended with status %s' % status)
        
def MIP_model(with_capacity=None, delete_facility=None, delete_facility_2=None, delete_facility_3=None):
    m = gp.Model('Facility Location Problem (Demand Satisfaction)')
    m.Params.LogToConsole = 0
    
    #Decision Variables
    y = m.addVars(num_facilities, vtype=GRB.BINARY, name='Select')
    x = m.addVars(CF, vtype=GRB.CONTINUOUS, name='Assign')

    #Constraints        
    m.addConstrs((x[(c,f)] <= demand[c]*y[f] for c in range(num_customers) for f in range(num_facilities)),
                 name='Serve if open')
    m.addConstrs((gp.quicksum(x[(c,f)] for f in range(num_facilities)) == demand[c] for c in range(num_customers)),
                 name='Demand satisfaction')
    
    if with_capacity==1:
        m.addConstrs((gp.quicksum(x[(c,f)] for c in range(num_customers)) <= capacity[f]*y[f] for f in range(num_facilities)),
             name='Capacity satisfaction')
    
    if delete_facility is not None:
        m.addConstr(y[delete_facility] == 0, name='Remove a candidate facility')
    if delete_facility_2 is not None:
        m.addConstr(y[delete_facility_2] == 0, name='Remove another candidate facility')
    if delete_facility_3 is not None:
        m.addConstr(y[delete_facility_3] == 0, name='Remove a 3rd candidate facility')

    #Objective Function
    m.setObjective(gp.quicksum(y[i]*opening_cost[i] for i in range(num_facilities))
                   + gp.quicksum(x[(c,f)]*DISTANCE[c][f]*cost_per_km for f in range(num_facilities) for c in range(num_customers)), GRB.MINIMIZE)
    
    # check_status(m.status)
    m.optimize()
    
    ## Solution for y[f]
    open_facilities = np.zeros(num_facilities)
    for facility in y.keys():
        if (abs(y[facility].x) > 1e-6):
            open_facilities[facility] = y[facility].x
        else:
            open_facilities[facility] = 0
    
    ## Solution for x[c,f]
    cost_fac = np.zeros((num_customers, num_facilities))
    shipments = np.zeros((num_customers, num_facilities))
    for c, f in x.keys():
        if (abs(x[c, f].x) > 1e-6):
            shipments[c,f] = x[c, f].x
            cost_fac[c,f] = 1
        else:
            shipments[c,f] = 0
            cost_fac[c,f] = 0
    
    return m.ObjVal, open_facilities, shipments, cost_fac

ObjVal, open_facilities, shipments, cost_fac = MIP_model()

In [ ]:
%%mnn widget --type plot --var customers_by_facilities --tab Report --header "Customers served by facility"
df = pd.DataFrame(shipments, columns=fac_key_list)
customers_by_facilities = px.bar(df.T.rename_axis('Facilities'), barmode='stack')

In [ ]:
%%mnn widget --type plot --var heatmap --tab Report --header "Heatmap of opened facilities"
heatmap = plt.figure(figsize=(12,3))
sns.set_style("whitegrid")
ax = sns.heatmap(shipments, linewidth=0.5, xticklabels=1, yticklabels=1, cmap="Purples")
plt.xticks(np.arange(num_facilities), labels=fac_key_list)
plt.xlabel("Facilities", fontsize = 20)
plt.ylabel("Customers", fontsize = 20)

In [ ]:
np.seterr('ignore')
shipments_cost = shipments*DISTANCE*cost_per_km
opencost_percus = np.where(np.isnan((cost_fac*opening_cost)/sum(cost_fac)), 0, (cost_fac*opening_cost)/sum(cost_fac))
cost_per_customer = (opencost_percus+shipments_cost)

In [ ]:
%%mnn widget --type plot --var cpc_chart --tab Report --header "Overall cost per customer"
df_total = pd.DataFrame(cost_per_customer)
cpc_chart = px.bar(df_total.rename_axis('Customers').set_axis(fac_key_list, axis=1))

In [ ]:
%%mnn widget --type plot --var fcpc_chart --tab Report --header "Facility cost per customer"
df_facility = pd.DataFrame(opencost_percus)
fcpc_chart = px.bar(df_facility.rename_axis('Customers').set_axis(fac_key_list, axis=1))